In [1]:
using Rx
using BenchmarkTools

In [2]:
function withRegularArray()
    values = [ i for i in 1:1000 ]
    keep = Array{Int, 1}()
    for value in values
         push!(keep, value ^ 2)
    end
    return keep
end

withRegularArray (generic function with 1 method)

In [5]:
struct KeepActor{D} <: NextActor{D}
    values::Array{D, 1}
    
    KeepActor{D}() where D = new(Array{D, 1}())
end

Rx.on_next!(actor::KeepActor{D}, data::D) where D = push!(actor.values, data)

In [4]:
@CreateMapOperator(Squared, (d) -> d ^ 2)

function withPregeneratedOperator()
    source = from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(source |> SquaredMapOperator{Int, Int}(), keep)
    return keep.values
end

withPregeneratedOperator (generic function with 1 method)

In [8]:
function withBaseMapOperator()
    source = from([ i for i in 1:1000 ])
    keep = KeepActor{Int}()
    subscribe!(source |> map(Int, Int, (d) -> d ^ 2), keep)
    return keep.values
end

withBaseMapOperator (generic function with 1 method)

In [6]:
@btime withRegularArray();

  6.576 μs (11 allocations: 24.33 KiB)


In [7]:
@btime withPregeneratedOperator();

  6.684 μs (15 allocations: 24.41 KiB)


In [9]:
@btime withBaseMapOperator();

  76.480 μs (2483 allocations: 62.98 KiB)
